# Homework 3 
## Barrett Johnson 

I have a catalog of repeating earthquakes, and I want to download seismic waveforms corresponding to these repeating earthquakes. However, when I look at the list of stations available in the seismic network, there are more than 6000. I do not want to download data from 6000 stations, so I want to filter only the seismic stations that are relevant for what I want to do with my waveforms.

In [1]:
# Address of the website to download data
url = 'http://ncedc.org/ftp/pub/doc/NC.info/NC.channel.summary.day'

In [2]:
# Useful Python modules
import numpy as np
import pandas as pd
import io
import pickle
import requests
from datetime import datetime, timedelta
from math import cos, sin, pi, sqrt

In [3]:
# Import the data from the website into a Python dataframe
s = requests.get(url).content
data = pd.read_csv(io.StringIO(s.decode('utf-8')), header=None, skiprows=2, sep='\s+', usecols=list(range(0, 13)))
data.columns = ['station', 'network', 'channel', 'location', 'rate', 'start_time', 'end_time', 'latitude', 'longitude', 'elevation', 'depth', 'dip', 'azimuth']

In [4]:
# Transform columns start_time and end_time into datetime format
startdate = pd.to_datetime(data['start_time'], format='%Y/%m/%d,%H:%M:%S')
data['start_time'] = startdate
# Avoid 'OutOfBoundsDatetime' error with year 3000
enddate = data['end_time'].str.replace('3000', '2025')
enddate = pd.to_datetime(enddate, format='%Y/%m/%d,%H:%M:%S')
data['end_time'] = enddate

In [5]:
### CHECK ###
# data

After discussing with my adviser, we decided than only the following channels are relevant for the work we want to do:

In [6]:
channels = ['BHE', 'BHN', 'BHZ', 'BH1', 'BH2', \
            'EHE', 'EHN', 'EHZ', 'EH1', 'EH2', \
            'HHE', 'HHN', 'HHZ', 'HH1', 'HH2', \
            'SHE', 'SHN', 'SHZ', 'SH1', 'SH2']

## First question

Filter the dataset to keep only the rows with the channels as defined above.

In [7]:
data = data.loc[data.channel.isin(channels)]

In [8]:
### CHECK ###
# data

My earthquake catalog starts on 2007/07/01 and ends on 2009/07/01. I am only interested in stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

## Second question

Filter the dataset to keep only stations that started recording before 2007/07/01 and ended recording after 2009/07/01.

In [9]:
t_start = datetime(2007,7,1,0,0,0)
t_end = datetime(2009,7,1,0,0,0)
data = data[(data['start_time'] <= t_start) & (data['end_time'] >= t_end)]

In [10]:
### CHECK ###
# data

I only want to keep the stations that are located less than 100 km from my repeating earthquakes. For stations farther away, the signal-to-noise ratio would be too low.

The earthquakes are located at latitude = 40.09 and longitude = -122.87. Here is a function to compute the distance from the station to the earthquakes, and to add a column distance to the dataset

In [11]:
a = 6378.136
e = 0.006694470
lat0 = 40.09000
lon0 = -122.87000
dx = (pi / 180.0) * a * cos(lat0 * pi / 180.0) / sqrt(1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0))
dy = (3.6 * pi / 648.0) * a * (1.0 - e * e) / ((1.0 - e * e * sin(lat0 * pi / 180.0) * sin(lat0 * pi / 180.0)) ** 1.5)
x = dx * (data['longitude'] - lon0)
y = dy * (data['latitude'] - lat0)
data['distance'] = np.sqrt(np.power(x, 2.0) + np.power(y, 2.0))

In [12]:
### CHECK ###
# data.describe()

## Third question

Filter the dataset to keep only stations that are less than 100 km from the earthquakes.

In [13]:
data = data.loc[data['distance'].values <100.0]

In [14]:
### CHECK ###
# data

Finally, I want to group the result such that the final result looks like:

|station|network|location|latitude|longitude |elevation|depth|distance |channel    |start_time         |end_time           |
|-------|-------|--------|--------|----------|---------|-----|---------|-----------|-------------------|-------------------|
|KBS 	|NC 	|-- 	 |39.91719|-123.59561|1120.0   |0.0  |64.720762|SHZ        |2002-10-17 00:00:00|2011-10-27 21:25:00|
|KCPB 	|NC 	|-- 	 |39.68631|-123.58242|1261.0   |0.0  |75.502041|HHZ,HHN,HHE|2006-10-18 00:08:00|2010-11-01 22:00:00|

I want all different channels to be grouped together, instead of having one row per channel. I also want to get the start_time end end_time for each station, instead of having it for each channel.

You can use the following function to group the channels together:

In [15]:
def f(x):
    """
    Concatenate channels
    """
    result = '%s' % ','.join(x)
#     print(result)
    result = list(set(result.split(',')))
#     print(result)
    result = '%s' % ','.join(result)
#     print(result)
    return result

In [16]:
data['channel'] = f(data.channel)

In [17]:
lat = data['latitude'].values
lo = data['longitude'].values
dis = data['distance'].values
nw = data['network'].values
ele = data['elevation'].values
loc = data['location'].values
z = data['depth'].values

## Fourth question

Use the pandas function agg to group the channels of a given station together, and compute the least recent start_time and the most recent end_time for each station.

In [18]:
data = data.groupby(['station', 'channel']).agg({'start_time':lambda x: min(x), 'end_time':lambda x: max(x)})

In [19]:
data = data.reset_index();

In [20]:
data['latitude']=lat 
data['longitude']=lo
data['distance']=dis
data['network']=nw
data['elevation']=ele
data['location']=loc
data['depth']=z

In [21]:
data = data[['station','network','location','latitude','longitude','elevation','depth','distance','channel','start_time','end_time']]

In [22]:
data.head(5)  ## Compare with example above, Same formatting ##

,station,network,location,latitude,longitude,elevation,depth,distance,channel,start_time,end_time
0,GBB,NC,--,39.80127,-122.34550,170.0,0.0,55.029997,"EHZ,SHZ",2000-12-06 18:38:00,2025-01-01 00:00:00
1,GCK,NC,--,39.54375,-122.43668,251.0,0.0,71.129241,"EHZ,SHZ",2000-06-06 21:58:00,2025-01-01 00:00:00
2,GFC,NC,--,39.32655,-122.28886,64.0,0.0,98.346307,"EHZ,SHZ",2001-04-03 23:25:00,2020-03-18 22:53:00
3,GHM,NC,--,39.49545,-122.93096,1456.0,0.0,66.387179,"EHZ,SHZ",1987-05-01 00:00:00,2025-01-01 00:00:00
4,GRO,NC,--,39.91684,-122.67117,1261.0,0.0,25.657089,"EHZ,SHZ",1990-12-13 23:30:00,2025-01-01 00:00:00


## Fifth question

How many stations are left in the dataset?

In [23]:
print('Number of stations remaining: ',len(data['station'].values))

Number of stations remaining:  26
